In [1]:
# 讀取資料，轉成dataframe
import pymongo
import pandas as pd

def connect_mongo():
    global collection
    client = pymongo.MongoClient("mongodb://192.168.71.128:27017/")
    # docker run --name mongo4 -v $(pwd)/mongodb_data:/data/db -d -p 27017:27017 --rm mongo:4.1
    # 透過VMware IP連進docker_mongodb
    db = client["db102"]  # Database Name
    collection = db["crawler_things"]   # Collection Name

if __name__ == "__main__":

    connect_mongo()  #呼叫連線資料庫函式
    cursor = collection.find()  # 抓出所有資料
    h_content =  pd.DataFrame(list(cursor))  #轉換成DataFrame
#     print(h_content)
# 若沒有mongdb, 也可以直接讀csv檔資料 h_content = pd.read_csv("beauty_health_lee_20190809.csv", encoding="utf-8")

In [2]:
# Kmeans 分群
import jieba 
from sklearn.feature_extraction.text import  TfidfVectorizer
from sklearn.cluster import KMeans
import re

def jieba_tokenize(text):
    text = re.sub("[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？、~@#￥%……&*（）【／】]+", "",text)  # 取代所有標點
    return jieba.lcut(text) 
 
 
tfidf_vectorizer = TfidfVectorizer(tokenizer=jieba_tokenize, lowercase=False)


tfidf_matrix = tfidf_vectorizer.fit_transform(h_content["content"])
 
num_clusters = 5
km_cluster = KMeans(n_clusters=num_clusters, max_iter=300, n_init=40, \
                    init='k-means++',n_jobs=-1)
'''
n_clusters: 指定K的值
max_iter: 对于单次初始值计算的最大迭代次数
n_init: 重新选择初始值的次数
init: 制定初始值选择的算法
n_jobs: 进程个数，为-1的时候是指默认跑满CPU
注意，这个对于单个初始值的计算始终只会使用单进程计算，
并行计算只是针对与不同初始值的计算。比如n_init=10，n_jobs=40, 
服务器上面有20个CPU可以开40个进程，最终只会开10个进程
'''
#返回各自文本的所被分配到的类索引
result = km_cluster.fit_predict(tfidf_matrix)
 
print("Predicting result: ", result)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\LEECHI~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.296 seconds.
Prefix dict has been built succesfully.


Predicting result:  [2 1 3 0 2 4 3 2 2 2 3 2 1 2 0 2 3 2 3 0 3 0 2 3 2 2 4 2 2 3 2 2 2 3 4 0 3
 2 1 0 3 2 3 2 0 2 2 1 3 3 3 2 3 0 4 3 3 2 2 4 1 3 3 0 3 3 1 4 2 3 2 2 3 3
 3 3 3 2 3 3 0 4 4 3 3 2 2 2 3 1 2 2 2 3 3 3 2 3 3 0 2 2 3 2 3 2 3 2 2 2 0
 2 1 0 2 2 2 3 2 0 2 1 3 2 2 2 4 0 3 1 0 0 1 3 2 2 2 2 2 2 2 1 3 2 2 2 3 3
 2 4 0 0 1 2 2 4 2 2 2 3 2 2 2 4 4 0 4 0 0 2 1 2 2 0 2 3 3 2 3 2 2 2 3 1 0
 3 2 0 0 2 2 3 0 3 1 2 2 4 2 3 1 3 3 2 4 2 2 2 3 2 2 3 2 2 1 2 1 2 3 1 3 4
 2 2 3 4 2 2 4 3 2 2 3 4 2 2 2 2 4 2 2 4 2 3 2 2 0 0 2 3 2 1 4 0 1 4 3 3 3
 3 4 2 4 2 1 4 4 2 2 2 2 2 4 1 2 3 2 2 2 1 3 2 2 1 2 1 2 2 3 2 2 1 2 3 4 2
 2 2 4 0 2 2 1 2 2 3 2 2 2 4 3 2 3 4 2 2 2 4 0 3 2 2 2 3 3 3 2 2 2 3 2 3 1
 2 2 3 3 2 2 3 0 3 3 0 0 3 2 4 4 2 4 2 2 1 3 2 4 2 3 2 2 4 2 4 4 0 2 2 2 2
 2 2 3 0 1 2 3 0 4 2 3 2 2 0 1 2 2 2 2 4 0 2 2 2 2 2 2 2 3 1 2 3 3 2 0 2 2
 3 2 2 2 2 2 2 3 0 2 0 2 2 2 2 2 2 2 2 4 3 2 2 2 2 2 2 2 3 1 2 2 2 2 0 2 2
 4 2 3 1 0 2 2 3 3 4 0 2 2 2 2 3 3 2 0 2 3 2 2 2 2 2 4 1 2 3 2 2 2 4 2 0 3
 2 2 

In [3]:
# 分群結果存成dataframe
y_train = pd.DataFrame(result,columns=["group"])
y_train
h_content = pd.concat([h_content, y_train], axis=1)  # 此行只要執行一次就好，不然會一直增加一樣的欄位

In [4]:
# 分舞群結果命名
trans = {
    "a":0,
    "b":1,
    "c":2,
    "d":3,
    "e":4
} 

In [5]:
# 找出每一群特徵關鍵字
import numpy as np
from jieba import analyse
# 引入TF-IDF關鍵詞抽取接口
def tfidf(text):
    return analyse.extract_tags(text)

def group_keyword_set(group_num):
    keywords = h_content["content"][h_content["group"] == group_num].apply(tfidf)  # 不用加小括號
    set_n = set()
    for i in keywords:
    #     print(i)
        for j in i:
            set_n.add(j)
    return set_n

for i in range(5):
    locals()[f"set{i}"] = group_keyword_set(i)

print("------第a組-----")
print(set0)
print("------第b組-----")
print(set1)
print("------第c組-----")
print(set2)
print("------第d組-----")
print(set3)
print("------第e組-----")
print(set4)

------第a組-----
{'腓腸', '一邊', '腫塊', '浸泡', '疏通', '擁有', '全臉', '幫助', '起來', '心臟', '上排', '牆壁', '推薦', '飯澤芳明', '同時', '三角', '屁股', '瘦腿', '足厥', '法令', '慢慢', '持續', '淋巴管', '過年', '透過', '168', '堵塞', '年齡', '排毒', '服部', '老舊', '拍拍', '腿部', '還有', '聲音', '指南', '蔬果', '緩慢', '甜食', 'LIFTY', '一個', '體幹', '進行動', '醫診', '廢物質', '可以', '各維持', '公室', '跪坐', '容易', '修飾', '惠貞', '膝蓋', '腦部', '坐著', '瘦大腿', '機制', '不覺', '抗老', '專業', '體內', '時間', '向後伸', '減少', '贅肉', '順暢', '合一', '交感', '暖和', '彩香', '宜民時', '調整', '下盤', '周圍', '體溫', '浴缸', '這個', '天氣', '網狀', '大家', '發出', '嘴型', '做出', '娛樂', '千葉', '美顏器', '顯現', '一組', '廢物', '左腳', '燃脂', '侯雅娟', '美顏術', '體操', '減緩', '分鐘', '模特', '森和世', '綜合', '凝露', '養成', '衰退', '側轉', '使用', '蛋白', '內側', '雙腿', '脾經', '簡單', '搭配', '伸直', '肌力', '肩膀', '站立', '深層', '強化', '貼布', '丹田', '堆積', '臀操', '節骨', '顯老', '專家', '代謝', '皺紋', '通過', '人體', '講師', '橘皮', '去除', 'Kimiko', '打造', '並且', '腳尖', '下半身', '彼拉', '凸起', '基礎', '手肘', '經絡', '鍛練', '覺得', '血管', '識地', '吸氣', '骨盆', '15', '扶牆', '田宜民', '一樣', '衣服', '帶動', '深呼吸', '活動', '越來', '還是', '辦公', '呼吸', '臀部', '粉類', 

In [6]:
from jieba import cut
import re

# s = "【早安健康／王彥文報導】愛穿洋裝的女孩們是否遇過這種情形？夏天到了，穿洋裝最舒服，但持續穿幾...  "
# string = re.sub("[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？、~@#￥%……&*（）【／】]+", "",s)  # 取代所有標點
# " ".join(cut(string))

# apply: Series.apply(函式名字)
def article_cut(s):
    string = re.sub("[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？、~@#￥%……&*（）【／】]+", "",s)  # 取代所有標點
    return " ".join(cut(string))
x_train = h_content["content"].apply(article_cut)  # 不用加小括號
x_train

0      早安 健康 王彥 文報 導愛穿 洋裝 的 女孩 們 是否 遇過 這種 情形 夏天 到 了 穿...
1      早安 健康 林昕潔 報導 你 也 有頭 皮癢 煩惱 嗎 每 當季節 或 環境 改變 或是 壓...
2      早安 健康 陳 彥甫 新北市 營養 師公會 常務 理事 多 吃 補 腎養 血 食物 少 吃 ...
3      早安 健康 朴惠貞 韓國 美容 專家 橘皮 組織 的 成因 到底 橘皮 組織 是 怎麼 生成...
4      早安 健康 林聖山 醫師 皮膚 科醫師 臉部 有 狀況 小心 蟲蟲 危機 毛囊 蠕形蟲症 D...
                             ...                        
514    最近 幾年 的 彩妝 趨勢 從 眼妝唇 妝 漸漸 發展 到 了 眉妝一 對 好看 的 眉毛 ...
515    坊 間 洗 髮 精百百種 該 怎麼 挑選 才能 找到 適合 自己 的 洗 髮 精 呢 植 髮...
516    天氣 濕熱 是 最 容易 出現 皮膚 病 的 時候 尤其 嬰幼兒 因為 皮膚 最為 細嫩 更...
517    我們 每天 吃 著賴以 為生 的 食物 勤奮 地 擦 保養品 卻 不 知道 吃 進去 的 食...
518    早安 健康 曾大容 報導頭 髮 是 影響 外表 的 關鍵 之一 為 了 維持 良好 的 儀表...
Name: content, Length: 519, dtype: object

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()

# 訓練: fit + transform  (將詞彙轉成欄位
x_train_vec = vec.fit_transform(x_train)

# # 測試: 不做fit 只做transform  (測試資料可能有訓練資料裡沒有的欄位，直接丟掉)
# x_test_vec = vec.transform(x_test)

In [8]:
vec.vocabulary_
# 看看詞語對應的欄位數字

{'早安': 9061,
 '健康': 2604,
 '王彥': 12292,
 '文報': 8923,
 '導愛穿': 6493,
 '洋裝': 11006,
 '女孩': 5828,
 '是否': 9135,
 '遇過': 18064,
 '這種': 17920,
 '情形': 7623,
 '夏天': 5376,
 '穿洋裝': 13887,
 '最舒服': 9485,
 '但持續': 2186,
 '幾天': 7032,
 '發現': 12970,
 '大腿': 5648,
 '內側': 2949,
 '兩端': 3078,
 '忽然': 7546,
 '一片': 705,
 '像是': 2740,
 '毛孔': 10482,
 '腫大': 15435,
 '紅疹': 14280,
 '有些': 9580,
 '出現': 3353,
 '時會': 9213,
 '感到': 7726,
 '不會': 1166,
 '但是': 2187,
 '忍著悶': 7530,
 '熱換': 12030,
 '穿長': 13894,
 '一點': 809,
 '褲子': 16616,
 '那片': 18441,
 '自然': 15726,
 '消失': 11215,
 '毛囊': 10480,
 '角化': 16754,
 '居多': 6686,
 '發期': 12946,
 '大多': 5570,
 '青春期': 19310,
 '其實': 3161,
 '可能': 4356,
 '任何': 2120,
 '年齡層': 7007,
 '除了': 19008,
 '以外': 2043,
 '手肘': 8039,
 '臀部': 15611,
 '臉頰': 15681,
 '背上': 15216,
 '症是': 12846,
 '怎麼樣': 7552,
 '一種': 726,
 '病症': 12834,
 '日本': 9045,
 '池袋': 10738,
 '西口': 16639,
 '貓頭鷹': 17236,
 '皮膚': 13196,
 '科診': 13744,
 '藤本': 16247,
 '智子': 9295,
 '院長': 19006,
 '解釋': 16788,
 '度角': 7086,
 '質化': 17318,
 '起因': 17427,
 '角質化': 16

In [9]:

from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(x_train_vec, y_train)  # 變成vec的 x_train

c:\users\leechiente\appdata\local\programs\python\python36\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [10]:
# 暫時沒有test資料
# from sklearn.metrics import accuracy_score

# pre = clf.predict(x_test_vec)  # 變成vec的 x_test
# accuracy_score(pre, y_test)

In [11]:
p = input("寫下您的困擾")
pcut = article_cut(p)
pvec = vec.transform([pcut])
print(pvec)


寫下您的困擾你好嗎
  (0, 2272)	1


In [12]:
reverse = {v:k for k, v in trans.items()}

pre = clf.predict(pvec)[0]
print("推薦您", reverse[pre], "的文章!")

推薦您 c 的文章!


In [13]:
# 結巴提取關鍵字
# import jieba.analyse
# i = 0
# for xt in x_train:
#     print(jieba.analyse.extract_tags(xt, topK=20, withWeight=True, allowPOS=()))
#     i += 1
#     if i > 10:
#         break

In [14]:

# import json
# import random
# import re

# import jieba
# import numpy as np
# from sklearn.feature_extraction.text import TfidfVectorizer

# jieba.load_userdict("./githubdict.txt")  # 加入自己的辭典